In [20]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pandas.tseries.offsets import DateOffset
from sklearn import metrics
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# the pandas webreader provides remote data access to apis
import pandas_datareader as webreader

date_today = "2020-01-01"
date_start = "2010-01-01"

# set the symbol to bitcoin-usd quotes
# for more symbols check yahoo.finance.com
symbol = "BTC-USD"

In [3]:
# now we will send the request to the yahoo finance api endpoint
df = webreader.DataReader(symbol, start=date_start, end=date_today, data_source="yahoo")
df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,21056800,457.334015
2014-09-18,456.859985,413.104004,456.859985,424.440002,34483200,424.440002
2014-09-19,427.834991,384.532013,424.102997,394.795990,37919700,394.795990
2014-09-20,423.295990,389.882996,394.673004,408.903992,36863600,408.903992
2014-09-21,412.425995,393.181000,408.084991,398.821014,26580100,398.821014


In [4]:
df['Pct Change'] = df['Close'].pct_change()

In [5]:
df['50 SMA'] = ta.SMA(df['Close'], timeperiod=50)
df['200 SMA'] = ta.SMA(df['Close'], timeperiod=200)
df['10 EMA'] = ta.EMA(df['Close'], timeperiod=10)
df['20 EMA'] = ta.EMA(df['Close'], timeperiod=20)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

In [7]:
df['Signal'] = 0
df.loc[(df['Pct Change'] >=0), 'Signal'] = 1
df.loc[(df['Pct Change'] <0), 'Signal'] = -1

In [8]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,Pct Change,50 SMA,200 SMA,10 EMA,20 EMA,RSI,Signal
Date,,,,,,,,,,,,,
2019-12-29,7513.948242,7279.865234,7317.647461,7422.652832,22445257702,7422.652832,0.014302,7594.035234,9321.269475,7303.683471,7313.282900,51.867873,1
2019-12-30,7454.824219,7276.308105,7420.272949,7292.995117,22874131672,7292.995117,-0.017468,7558.784609,9316.579832,7301.740134,7311.350730,47.833501,-1
2019-12-31,7335.290039,7169.777832,7294.438965,7193.599121,21167946112,7193.599121,-0.013629,7527.500830,9309.078662,7282.078132,7300.136291,44.947238,-1
2020-01-01,7254.330566,7174.944336,7194.892090,7200.174316,18565664997,7200.174316,0.000914,7495.191074,9300.887659,7267.186529,7290.616103,45.182876,1
2020-01-02,7212.155273,6935.270020,7202.551270,6985.470215,20802083465,6985.470215,-0.029819,7458.735225,9290.842568,7215.965381,7261.554590,39.271849,-1


In [11]:
X = df[['Close', 'Volume', 'Pct Change', '50 SMA', '200 SMA', 'RSI']].shift().dropna()
y = df['Signal']

In [14]:
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=12)

In [15]:
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

In [16]:
X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]

In [17]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
svm_model = SVC()
svm_model = svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [21]:
svm_testing_report = classification_report(y_test, svm_pred)
print(svm_testing_report)

              precision    recall  f1-score   support

          -1       0.29      0.01      0.01       606
           1       0.56      0.99      0.71       761

    accuracy                           0.55      1367
   macro avg       0.42      0.50      0.36      1367
weighted avg       0.44      0.55      0.40      1367

